# 📊 Exercícios Práticos: Teoria da Amostragem

Este notebook contém exercícios práticos sobre tipos de amostragem, viés e técnicas de seleção de amostras.

## Referências Teóricas

Os exercícios são baseados em:
- **Cochran, W.G. (1977).** *Sampling Techniques*. 3. ed. New York: Wiley.
- **Thompson, S.K. (2012).** *Sampling*. 3. ed. Hoboken: Wiley.
- **Lohr, S.L. (2019).** *Sampling: Design and Analysis*. 2. ed. Boca Raton: CRC Press.

In [ ]:
# Bibliotecas necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Configurações
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

## Exercício 1: Amostragem Aleatória Simples (AAS)

### Problema
Uma universidade tem 10.000 alunos e deseja estimar o tempo médio que os estudantes gastam estudando por semana. Você precisa selecionar uma amostra aleatória simples de 100 alunos.

### Tarefas
1. Simule uma população de 10.000 alunos com horas de estudo seguindo uma distribuição normal (μ=20, σ=5)
2. Selecione 3 amostras aleatórias de 100 alunos cada
3. Calcule a média de cada amostra
4. Compare com a média populacional
5. Calcule o erro amostral para cada amostra

### Fundamento Teórico
Segundo **Cochran (1977)**, na AAS, cada elemento tem probabilidade igual de seleção: $P(i) = n/N$, onde $n$ é o tamanho da amostra e $N$ é o tamanho da população.

In [ ]:
# Solução do Exercício 1

# 1. Criar população
N = 10000  # Tamanho da população
mu_pop = 20  # Média populacional
sigma_pop = 5  # Desvio padrão populacional

populacao = np.random.normal(mu_pop, sigma_pop, N)
media_pop = populacao.mean()

print(f"População:")
print(f"  Tamanho: {N}")
print(f"  Média real: {media_pop:.2f} horas")
print(f"  Desvio padrão real: {populacao.std():.2f} horas")

# 2 e 3. Selecionar amostras e calcular médias
n = 100  # Tamanho da amostra
num_amostras = 3

print(f"\nAmostras Aleatórias Simples (n={n}):")
for i in range(num_amostras):
    amostra = np.random.choice(populacao, size=n, replace=False)
    media_amostra = amostra.mean()
    erro = abs(media_amostra - media_pop)
    
    print(f"\nAmostra {i+1}:")
    print(f"  Média: {media_amostra:.2f} horas")
    print(f"  Erro amostral: {erro:.2f} horas")
    print(f"  Erro relativo: {(erro/media_pop)*100:.2f}%")

## Exercício 2: Amostragem Estratificada

### Problema
A mesma universidade tem alunos de 4 cursos diferentes com características distintas:
- Engenharia: 3000 alunos (média: 25h, σ=4)
- Medicina: 2000 alunos (média: 30h, σ=6)
- Humanas: 3000 alunos (média: 15h, σ=5)
- Artes: 2000 alunos (média: 18h, σ=7)

### Tarefas
1. Compare amostragem aleatória simples vs. estratificada proporcional
2. Calcule o erro de cada método
3. Demonstre quando a estratificação é mais eficiente

### Fundamento Teórico
**Lohr (2019)** demonstra que a amostragem estratificada reduz a variância quando os estratos são homogêneos internamente e heterogêneos entre si. A variância do estimador estratificado é:

$$V(\bar{y}_{st}) = \sum_{h=1}^{L} W_h^2 \frac{S_h^2}{n_h}$$

onde $W_h = N_h/N$ é o peso do estrato.

In [ ]:
# Solução do Exercício 2

# Criar população estratificada
estratos = {
    'Engenharia': {'N': 3000, 'media': 25, 'sigma': 4},
    'Medicina': {'N': 2000, 'media': 30, 'sigma': 6},
    'Humanas': {'N': 3000, 'media': 15, 'sigma': 5},
    'Artes': {'N': 2000, 'media': 18, 'sigma': 7}
}

# Gerar população estratificada
pop_estratificada = []
for curso, params in estratos.items():
    dados = np.random.normal(params['media'], params['sigma'], params['N'])
    df_curso = pd.DataFrame({
        'horas': dados,
        'curso': curso
    })
    pop_estratificada.append(df_curso)

df_pop = pd.concat(pop_estratificada, ignore_index=True)
media_pop_real = df_pop['horas'].mean()

print("População Total:")
print(f"  Tamanho: {len(df_pop)}")
print(f"  Média: {media_pop_real:.2f} horas\n")

# Estatísticas por estrato
print("Estatísticas por Curso:")
print(df_pop.groupby('curso')['horas'].agg(['count', 'mean', 'std']).round(2))

# 1. Amostragem Aleatória Simples
n_total = 200
amostra_aas = df_pop.sample(n=n_total, replace=False)
media_aas = amostra_aas['horas'].mean()
erro_aas = abs(media_aas - media_pop_real)

print(f"\n--- Amostragem Aleatória Simples (n={n_total}) ---")
print(f"Média estimada: {media_aas:.2f} horas")
print(f"Erro: {erro_aas:.2f} horas")

# 2. Amostragem Estratificada Proporcional
amostras_estratificadas = []
for curso in estratos.keys():
    df_curso = df_pop[df_pop['curso'] == curso]
    prop = len(df_curso) / len(df_pop)
    n_estrato = int(n_total * prop)
    amostra_estrato = df_curso.sample(n=n_estrato, replace=False)
    amostras_estratificadas.append(amostra_estrato)

amostra_estratificada = pd.concat(amostras_estratificadas)
media_estratificada = amostra_estratificada['horas'].mean()
erro_estratificada = abs(media_estratificada - media_pop_real)

print(f"\n--- Amostragem Estratificada Proporcional (n={n_total}) ---")
print(f"Média estimada: {media_estratificada:.2f} horas")
print(f"Erro: {erro_estratificada:.2f} horas")

# Comparação
print(f"\n--- Comparação ---")
print(f"Redução de erro: {((erro_aas - erro_estratificada)/erro_aas)*100:.1f}%")
print(f"Eficiência relativa: {(erro_aas/erro_estratificada):.2f}x")

## Exercício 3: Viés de Seleção

### Problema
Demonstrar o impacto do viés de seleção em uma pesquisa online sobre satisfação com serviços de streaming.

### Cenário
- População: 10.000 usuários
- 60% estão satisfeitos (nota média: 8.0)
- 40% estão insatisfeitos (nota média: 3.0)
- Viés: Usuários insatisfeitos têm 3x mais probabilidade de responder

### Fundamento Teórico
**Groves et al. (2009)** em *Survey Methodology* demonstram que o viés de não-resposta pode ser expresso como:

$$B(\bar{y}) = (\bar{y}_r - \bar{y}_{nr}) \times (1 - r)$$

onde $\bar{y}_r$ é a média dos respondentes, $\bar{y}_{nr}$ dos não-respondentes, e $r$ é a taxa de resposta.

In [ ]:
# Solução do Exercício 3

# População
n_satisfeitos = 6000
n_insatisfeitos = 4000

pop_satisfeitos = np.random.normal(8.0, 1.0, n_satisfeitos)
pop_insatisfeitos = np.random.normal(3.0, 1.0, n_insatisfeitos)

populacao_total = np.concatenate([pop_satisfeitos, pop_insatisfeitos])
media_real = populacao_total.mean()

print(f"Média populacional real: {media_real:.2f}")

# Simulação SEM viés (amostra aleatória)
amostra_sem_vies = np.random.choice(populacao_total, size=500, replace=False)
media_sem_vies = amostra_sem_vies.mean()

print(f"\nAmostragem SEM viés:")
print(f"  Média estimada: {media_sem_vies:.2f}")
print(f"  Erro: {abs(media_sem_vies - media_real):.2f}")

# Simulação COM viés (insatisfeitos têm 3x mais chance de responder)
# Probabilidades de resposta
prob_satisfeito = 0.25
prob_insatisfeito = 0.75  # 3x mais

# Respondentes satisfeitos e insatisfeitos
n_resp_satisfeitos = int(n_satisfeitos * prob_satisfeito * 0.083)  # ajuste para obter ~500 total
n_resp_insatisfeitos = int(n_insatisfeitos * prob_insatisfeito * 0.125)

resp_satisfeitos = np.random.choice(pop_satisfeitos, size=n_resp_satisfeitos, replace=False)
resp_insatisfeitos = np.random.choice(pop_insatisfeitos, size=n_resp_insatisfeitos, replace=False)

amostra_com_vies = np.concatenate([resp_satisfeitos, resp_insatisfeitos])
media_com_vies = amostra_com_vies.mean()

print(f"\nAmostragem COM viés de não-resposta:")
print(f"  Respondentes satisfeitos: {n_resp_satisfeitos}")
print(f"  Respondentes insatisfeitos: {n_resp_insatisfeitos}")
print(f"  Média estimada: {media_com_vies:.2f}")
print(f"  Erro: {abs(media_com_vies - media_real):.2f}")
print(f"  Viés: {media_com_vies - media_real:.2f}")

# Visualização
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# População
axes[0].hist(populacao_total, bins=30, alpha=0.7, color='gray', edgecolor='black')
axes[0].axvline(media_real, color='red', linestyle='--', linewidth=2, label=f'Média real: {media_real:.2f}')
axes[0].set_title('População')
axes[0].set_xlabel('Satisfação (0-10)')
axes[0].set_ylabel('Frequência')
axes[0].legend()

# Sem viés
axes[1].hist(amostra_sem_vies, bins=20, alpha=0.7, color='green', edgecolor='black')
axes[1].axvline(media_sem_vies, color='blue', linestyle='--', linewidth=2, label=f'Média: {media_sem_vies:.2f}')
axes[1].axvline(media_real, color='red', linestyle='--', linewidth=2, label=f'Real: {media_real:.2f}')
axes[1].set_title('Amostra SEM Viés')
axes[1].set_xlabel('Satisfação (0-10)')
axes[1].legend()

# Com viés
axes[2].hist(amostra_com_vies, bins=20, alpha=0.7, color='orange', edgecolor='black')
axes[2].axvline(media_com_vies, color='blue', linestyle='--', linewidth=2, label=f'Média: {media_com_vies:.2f}')
axes[2].axvline(media_real, color='red', linestyle='--', linewidth=2, label=f'Real: {media_real:.2f}')
axes[2].set_title('Amostra COM Viés')
axes[2].set_xlabel('Satisfação (0-10)')
axes[2].legend()

plt.tight_layout()
plt.show()

print(f"\n--- Impacto do Viés ---")
print(f"Erro adicional devido ao viés: {abs(media_com_vies - media_real) - abs(media_sem_vies - media_real):.2f} pontos")

## Exercício 4: Cálculo de Tamanho Amostral

### Problema
Calcular o tamanho amostral necessário para diferentes cenários de pesquisa.

### Fundamento Teórico
Segundo **Cochran (1977)**, para estimar uma média populacional com precisão desejada, o tamanho amostral é:

$$n = \frac{N \cdot Z^2 \cdot \sigma^2}{(N-1) \cdot E^2 + Z^2 \cdot \sigma^2}$$

onde:
- $N$ = tamanho da população
- $Z$ = valor crítico (1.96 para 95% de confiança)
- $\sigma^2$ = variância populacional
- $E$ = margem de erro desejada

In [ ]:
# Solução do Exercício 4

def calcular_tamanho_amostra(N, sigma, E, confianca=0.95):
    """
    Calcula tamanho amostral para estimativa de média.
    
    Parâmetros:
    -----------
    N : int
        Tamanho da população
    sigma : float
        Desvio padrão populacional
    E : float
        Margem de erro desejada
    confianca : float
        Nível de confiança (default=0.95)
    
    Retorna:
    --------
    n : int
        Tamanho amostral necessário
    
    Referência: Cochran (1977), Sampling Techniques, 3rd ed.
    """
    # Valor crítico Z
    Z = stats.norm.ppf((1 + confianca) / 2)
    
    # Fórmula para população finita
    numerador = N * (Z**2) * (sigma**2)
    denominador = (N - 1) * (E**2) + (Z**2) * (sigma**2)
    
    n = numerador / denominador
    
    return int(np.ceil(n))

# Cenários
cenarios = [
    {'nome': 'Pesquisa de Satisfação', 'N': 5000, 'sigma': 2.0, 'E': 0.3},
    {'nome': 'Estudo de Renda', 'N': 50000, 'sigma': 500, 'E': 50},
    {'nome': 'Avaliação de Produto', 'N': 1000, 'sigma': 1.5, 'E': 0.2},
    {'nome': 'Pesquisa Eleitoral', 'N': 100000, 'sigma': 0.5, 'E': 0.03}
]

print("Cálculo de Tamanho Amostral\n")
print("=" * 80)

resultados = []
for cenario in cenarios:
    n = calcular_tamanho_amostra(cenario['N'], cenario['sigma'], cenario['E'])
    prop = (n / cenario['N']) * 100
    
    print(f"\n{cenario['nome']}:")
    print(f"  População (N): {cenario['N']:,}")
    print(f"  Desvio padrão (σ): {cenario['sigma']}")
    print(f"  Margem de erro (E): {cenario['E']}")
    print(f"  Tamanho amostral necessário: {n:,} ({prop:.1f}% da população)")
    
    resultados.append({
        'Cenário': cenario['nome'],
        'N': cenario['N'],
        'n': n,
        'Proporção (%)': round(prop, 2)
    })

# Tabela resumo
print("\n" + "=" * 80)
print("\nResumo:")
df_resultados = pd.DataFrame(resultados)
print(df_resultados.to_string(index=False))

## Exercício 5: Amostragem Sistemática

### Problema
Implementar e comparar amostragem sistemática com amostragem aleatória simples.

### Fundamento Teórico
**Thompson (2012)** explica que na amostragem sistemática, selecionamos cada k-ésimo elemento da população, onde:

$$k = \frac{N}{n}$$

Este método é eficiente quando a população não tem periodicidade. A vantagem é a facilidade de implementação.

In [ ]:
# Solução do Exercício 5

# População ordenada (ex: lista de clientes por ordem de cadastro)
N = 1000
n = 50

# População com valores variados
np.random.seed(42)
populacao = np.random.gamma(shape=2, scale=50, size=N)  # Distribuição assimétrica
media_pop = populacao.mean()

print(f"População: N = {N}, média = {media_pop:.2f}\n")

# Amostragem Aleatória Simples
indices_aas = np.random.choice(N, size=n, replace=False)
amostra_aas = populacao[indices_aas]
media_aas = amostra_aas.mean()

print(f"Amostragem Aleatória Simples:")
print(f"  Média: {media_aas:.2f}")
print(f"  Erro: {abs(media_aas - media_pop):.2f}\n")

# Amostragem Sistemática
k = N // n  # Intervalo de seleção
inicio = np.random.randint(0, k)  # Ponto de partida aleatório
indices_sistematica = np.arange(inicio, N, k)[:n]
amostra_sistematica = populacao[indices_sistematica]
media_sistematica = amostra_sistematica.mean()

print(f"Amostragem Sistemática (k={k}, início={inicio}):")
print(f"  Média: {media_sistematica:.2f}")
print(f"  Erro: {abs(media_sistematica - media_pop):.2f}\n")

# Simulação com múltiplas amostras
n_simulacoes = 1000
erros_aas = []
erros_sistematica = []

for _ in range(n_simulacoes):
    # AAS
    amostra = np.random.choice(populacao, size=n, replace=False)
    erros_aas.append(abs(amostra.mean() - media_pop))
    
    # Sistemática
    inicio = np.random.randint(0, k)
    indices = np.arange(inicio, N, k)[:n]
    amostra = populacao[indices]
    erros_sistematica.append(abs(amostra.mean() - media_pop))

# Comparação
print(f"--- Comparação ({n_simulacoes} simulações) ---")
print(f"Erro médio AAS: {np.mean(erros_aas):.2f}")
print(f"Erro médio Sistemática: {np.mean(erros_sistematica):.2f}")
print(f"Desvio padrão AAS: {np.std(erros_aas):.2f}")
print(f"Desvio padrão Sistemática: {np.std(erros_sistematica):.2f}")

# Visualização
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(erros_aas, bins=30, alpha=0.7, color='blue', edgecolor='black', label='AAS')
axes[0].hist(erros_sistematica, bins=30, alpha=0.7, color='orange', edgecolor='black', label='Sistemática')
axes[0].set_xlabel('Erro Absoluto')
axes[0].set_ylabel('Frequência')
axes[0].set_title('Distribuição dos Erros')
axes[0].legend()

axes[1].boxplot([erros_aas, erros_sistematica], labels=['AAS', 'Sistemática'])
axes[1].set_ylabel('Erro Absoluto')
axes[1].set_title('Comparação dos Erros')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 📚 Resumo e Conclusões

### Principais Aprendizados

1. **Amostragem Aleatória Simples (AAS)**:
   - Método fundamental onde cada elemento tem probabilidade igual de seleção
   - Referência: **Cochran (1977)**

2. **Amostragem Estratificada**:
   - Reduz variância quando estratos são homogêneos internamente
   - Mais eficiente que AAS em populações heterogêneas
   - Referência: **Lohr (2019)**

3. **Viés de Seleção**:
   - Pode distorcer severamente os resultados
   - Viés de não-resposta é especialmente problemático
   - Referência: **Groves et al. (2009)**

4. **Tamanho Amostral**:
   - Depende da variabilidade populacional, margem de erro e nível de confiança
   - Para populações grandes, o tamanho amostral se estabiliza
   - Referência: **Cochran (1977)**

5. **Amostragem Sistemática**:
   - Eficiente e fácil de implementar
   - Cuidado com periodicidade na população
   - Referência: **Thompson (2012)**

### Referências Bibliográficas

**COCHRAN, William G.** *Sampling Techniques*. 3. ed. New York: John Wiley & Sons, 1977.
- Obra clássica sobre teoria de amostragem

**LOHR, Sharon L.** *Sampling: Design and Analysis*. 2. ed. Boston: Brooks/Cole, 2019.
- Texto moderno com aplicações práticas

**THOMPSON, Steven K.** *Sampling*. 3. ed. Hoboken: Wiley, 2012.
- Abordagem contemporânea de métodos de amostragem

**GROVES, Robert M. et al.** *Survey Methodology*. 2. ed. Hoboken: Wiley, 2009.
- Metodologia completa de pesquisas por amostragem

**SÄRNDAL, Carl-Erik; SWENSSON, Bengt; WRETMAN, Jan.** *Model Assisted Survey Sampling*. New York: Springer, 1992.
- Abordagem avançada usando modelos estatísticos